# Pre-processing: multiple sessions at the same time

Version 4 · Last update: 09/10/2024

## Step 0
* Initialisation
* Select subjects

<span style="color:#99cc00">Fully automatic step, approx. 1 sec per subject</span><br>
<span style="color:#ff0000">**This step must be executed every time!**</span>

In [6]:
# Put in the following arrays the subjects you want to pre-process, subset of the keys from subjects_paths
selected_subjects = ["subject_01a", "subject_01b", "subject_02a", "subject_02b"]

# Navigate in the OS and call folders
import os
import os.path as op

# Get the username automatically
import getpass

# Perform plots
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np

# Date and time
from datetime import datetime as dt
from datetime import timedelta as td

# MEG processing
import mne
from mne_bids import (
    write_raw_bids,
    read_raw_bids,
    BIDSPath,
    write_meg_calibration,
    write_meg_crosstalk,
    mark_channels
)

# Configuration
user = getpass.getuser()
path_data = op.join('/export/home', user, 'lab/MEG_EXPERIMENTS/EXPERIMENT_NAME/DATA/FIF/')

# Subjects (all of the subjects from the experiment)
subjects_paths = {"subject_01a": 'path_to_subject_01/session_01/block_01',
                  "subject_01b": 'path_to_subject_01/session_02/block_01',
                  "subject_02a": 'path_to_subject_02/session_01/block_01',
                  "subject_02b": 'path_to_subject_02/session_02/block_01'}

# Define the raw BIDS output path
path_output = op.join('/export', 'home', user, 'public', 'MEGtrain')
path_bids = op.join(path_output, 'bids')

# Calibration and crosstalk files 
path_maxfilter_parameters = op.join(path_output, "scripts", "maxfilter_parameters")
fine_cal_file = op.join(path_maxfilter_parameters, "sss_cal_3049.dat")
crosstalk_file = op.join(path_maxfilter_parameters, "ct_sparse.fif")

# Define the preprocessing output path
path_preprocessing = op.join(path_output, "derivatives", "Preprocessing")

# Define the eelbrain path
path_eelbrain_meg = op.join(path_output, 'eelbrain', 'meg')
os.makedirs(path_eelbrain_meg, exist_ok=True)

# Empty room
path_empty_room = op.join('/export/home', user, 'lab/MEG_EXPERIMENTS/EMPTYROOM/DATA/')

class Subject(object):

    def __init__(self, subject):
        self.name = subject

        if subject not in subjects_paths.keys():
            raise Exception("Subject name not found.")
        self.fif = subjects_paths[subject] + ".fif"
        self.number = subject[8:10]
        self.session = "02" if subjects_paths[subject].split("/")[0][-2:] == "_2" else "01"
        self.visit = " 6mt" if subject[10] == "b" else ""
        self.raw_fif_name = op.join(path_data, self.fif)
        self.sub = "sub-" + self.number

        self.path_derivatives = BIDSPath(subject = self.number, session = self.session, task = 'track', run = '01', suffix = "meg", root = path_preprocessing)
        self.path_bids_raw = BIDSPath(subject = self.number, session = self.session, task = 'track', run = '01', suffix = "meg", root = path_bids)
        self.path_bids_empty_room = BIDSPath(subject = self.number, session = self.session, task = 'emptyroom', run = '01', suffix = "meg", root = path_bids)
        self.path_derivatives_er = BIDSPath(subject = self.number, session = self.session, task = 'emptyroom', run = '01', suffix = "meg", root = path_preprocessing)

        self.path_downsampled_sss = op.join(self.path_derivatives.directory, self.path_derivatives.basename.replace("meg", "sss") + ".fif")
        self.path_ica_solution = op.join(self.path_derivatives.directory, self.path_derivatives.basename.replace("meg", "ica_solution") + ".fif")
        self.path_ica = op.join(self.path_derivatives.directory, self.path_derivatives.basename.replace("meg", "ica") + ".fif")
        self.path_eelbrain = op.join(path_eelbrain_meg, self.sub, self.sub + "_track" + self.visit + "-raw.fif")
        self.path_downsampled_sss_er = op.join(self.path_derivatives_er.directory, self.path_derivatives_er.basename.replace("meg", "sss") + ".fif")
        self.path_eelbrain_er = op.join(path_eelbrain_meg, self.sub, self.sub + "_emptyroom" + self.visit + "-raw.fif")

        os.makedirs(op.join(path_eelbrain_meg, self.sub), exist_ok=True)

    def print_name(self):
        s = "#"
        print("")
        print(s * (len(self.name) + 10))
        print(s + s + "   " + self.name.upper() + "   " + s + s)
        print(s * (len(self.name) + 10))
        print("")

all_subjects = {}
for subject_name in subjects_paths.keys():
    all_subjects[subject_name] = Subject(subject_name)

subjects = []
for subject_name in selected_subjects:
    subjects.append(all_subjects[subject_name])

# PARAMETERS
show_timings = True

# Define the manuel bad channels for each subject - will be ADDED to the automatic bad channels (step 2)
bad_channels_subject = {"subject_01a": ["MEG2623"],
                        "subject_01b": ["MEG1912"],
                        "subject_02a": ["MEG2623"],
                        "subject_02b": ["MEG0312", "MEG0711"]}

# EOG and ECG channels for each subject (step 4)
default_channels = {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"}
eog_ecg_channels = {"subject_01a": {"HEOG": "EOG061", "VEOG": "EEG005", "ECG": "ECG063"},
                    "subject_01b": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"},
                    "subject_02a": {"HEOG": "EOG061", "VEOG": "EEG005", "ECG": "ECG063"},
                    "subject_02b": {"HEOG": "EOG061", "VEOG": "EOG062", "ECG": "ECG063"}}
# eog_ecg_channels = {}

# ICA components to exclude - will REPLACE the components to exclude (steps 5 and 6)
ica_components = {"subject_01a": [15, 16, 19],
                  "subject_01b": [0, 1, 4],
                  "subject_02a": [5, 14],
                  "subject_02b": [5, 13, 15]}

# Choose what to plot (step 5)
plot_components_locations = True
plot_components_time_course = True
plot_ecg_eog = True
plot_components_scores = False
plot_explained_variance = False
plot_excluded_components_and_original = False
plot_overlay = False

# Choose if to save the figures in step 6
save_figures = True

# Define the manuel bad channels for each subject - will be ADDED to the automatic bad channels (step 7)
bad_channels_subject_er = {"subject_01a": ["MEG2623", "MEG1122"],
                           "subject_01b": ["MEG1741", "MEG1833"],
                           "subject_02a": ["MEG2623", "MEG1122"],
                           "subject_02b": ["MEG0143"]}

## Step 1
* BIDS creation
* Automatically detect bad channels (saved into the BIDS file)

<span style="color:#99cc00">Fully automatic step, approx. 10 min per subject</span><br>
<span style="color:#ff0000">Check that you do not have any "merge.fif" file in your subjects folders!</span>

In [ ]:
timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    os.makedirs(subject.path_derivatives.directory, exist_ok = True)

    # Read the FIF file
    print("Reading original FIF...")
    t_step = dt.now()
    raw_fif = mne.io.read_raw_fif(subject.raw_fif_name, preload = False, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Write the BIDS files
    print("Writing BIDS...")
    t_step = dt.now()
    write_raw_bids(raw = raw_fif, bids_path = subject.path_bids_raw, overwrite = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_fif
    print("Deleting raw_fif...")
    t_step = dt.now()
    del raw_fif
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")
    
    # Write calibration files in BIDS format
    print("Writing calibration files in BIDS format...")
    t_step = dt.now()
    write_meg_calibration(fine_cal_file, subject.path_bids_raw, verbose = False)
    write_meg_crosstalk(crosstalk_file, subject.path_bids_raw, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Open the BIDS data
    print("Opening BIDS data...")
    t_step = dt.now()
    raw_bids = read_raw_bids(subject.path_bids_raw, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Define the "bads" channels to being empty
    raw_bids.info['bads'] = []

    # Run the automatic detection
    print("Running the automatic detection of bad channels...")
    t_step = dt.now()
    auto_noisy_channels, auto_flat_channels, auto_scores = mne.preprocessing.find_bad_channels_maxwell(
        raw_bids,  # Raw data to process
        cross_talk = crosstalk_file, # MEG crosstalk file
        calibration = fine_cal_file,  # MEG fine-calibration file
        return_scores = True, 
        duration = 30,  # Duration of each window (in seconds)
        min_count = 10,  # Number of window appearances to be counted as bad channel
        verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")
    
    print(f"\tFlat channels: {str(auto_flat_channels)}")
    print(f"\tBad channels: {str(auto_noisy_channels)}")

    raw_bids.info['bads'] = auto_flat_channels + auto_noisy_channels

    print("Writing the BIDS with the detected bad channels info...")
    t_step = dt.now()
    mark_channels(subject.path_bids_raw, ch_names = raw_bids.info['bads'], status = "bad", verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_bids
    print("Deleting raw_bids...")
    t_step = dt.now()
    del raw_bids
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")

## Step 2

* Manual channel detection: click on the channels you want to remove.

<span style="color:#ffcc00">Manual step</span><br>
If the dictionary `bad_channels_subject` in step 0 contains an entry with the subject name, this step becomes automatic.

In [ ]:
subject = all_subjects["subject_07a"]
raw_bids = mne.io.read_raw_fif(subject.raw_fif_name, verbose = False)
raw_bids.plot(lowpass = 30, highpass = 1, overview_mode = "hidden", title = subject.name, block = True)
channel = raw_bids["STI101"]
trigger_values = channel[0][0]
trigger_values
previous_value = None
with open(path_output+f"/triggers/triggers_{subject.name}.txt", "w") as f:
    for i in range(len(trigger_values)):
        if trigger_values[i] != previous_value:
            f.write(f"{i}\t{trigger_values[i]}\n")
            previous_value = trigger_values[i]

In [ ]:
mne.viz.set_browser_backend("qt")

timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    # Open the BIDS data
    print("Opening BIDS data...")
    t_step = dt.now()
    raw_bids = read_raw_bids(subject.path_bids_raw, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    print("Automatically detected bad channels: ")
    print(", ".join(raw_bids.info['bads']))

    # We skip it if we saved the bad channels in bad_channels_subject
    if subject.name in bad_channels_subject.keys():
        raw_bids.info['bads'] += bad_channels_subject[subject.name]

        print("Bad channels after adding the ones saved in bad_channels_subject: ")
        print(", ".join(raw_bids.info['bads']))

    else:
        print("Opening the GUI to manually mark bad channels...")
        t_step = dt.now()
        automatic_channels = []
        for channel in raw_bids.info['bads']:
            automatic_channels.append(channel)
        
        raw_bids.plot(lowpass = 30, highpass = 1, overview_mode = "hidden", title = subject.name, block = True)

        print("Manually added channels: ")
        manual_channels = list(set(raw_bids.info['bads']) - set(automatic_channels))
        print(", ".join(manual_channels))

        print("Bad channels after manual processing: ")
        print(", ".join(raw_bids.info['bads']))

    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Write the BIDS data
    print("Writing the BIDS with the detected bad channels info...")
    t_step = dt.now()
    mark_channels(subject.path_bids_raw, ch_names = raw_bids.info['bads'], status = "bad", verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_bids
    print("Deleting raw_bids...")
    t_step = dt.now()
    del raw_bids
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")

## Step 3
* Compute the time-varying head positions
* Run the Maxfilter
* Save the filtered data
* Generate and fit the ICA
* Save the ICA

<span style="color:#99cc00">Fully automatic step, approx. 2 to 6 hrs per subject (depending on the server's load)</span><br>

In [ ]:
#mne.set_config("MNE_USE_NUMBA", "true")

timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    # Open the BIDS data
    print("Opening BIDS data...")
    t_step = dt.now()
    raw_bids = read_raw_bids(subject.path_bids_raw, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Extract continuous head position indicators (cHPI)
    print("Computing time-varying cHPI amplitudes...")
    t_step = dt.now()
    chpi_amplitudes = mne.chpi.compute_chpi_amplitudes(raw_bids, t_window = 0.5, t_step_min = 0.1, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Estimate the coils locations
    print("Computing locations of each cHPI coils over time...")
    t_step = dt.now()
    chpi_locs = mne.chpi.compute_chpi_locs(raw_bids.info, chpi_amplitudes, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Estimate the head positions
    print("Computing time-varying head positions...")
    t_step = dt.now()
    head_pos = mne.chpi.compute_head_pos(raw_bids.info, chpi_locs, gof_limit = 0.95, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Save the head positions
    print("Saving continuous head positions...")
    t_step = dt.now()
    path_pos = op.join(subject.path_derivatives.directory, subject.path_derivatives.basename + ".pos")
    mne.chpi.write_head_pos(path_pos, head_pos)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Run the Maxfilter
    print("Running the Maxfilter...")
    t_step = dt.now()
    raw_sss = mne.preprocessing.maxwell_filter(
        raw_bids, 
        cross_talk = crosstalk_file,  # MEG crosstalk file
        calibration = fine_cal_file,  # MEG fine-calibration file
        head_pos = head_pos,  # Head positions
        st_duration = 10,  # Duration of the buffer (in seconds) 
        st_correlation = 0.98,  # Correlation limit between inner and outer subspaces used to reject overlapping intersecting inner/outer signals during spatiotemporal SSS.*
        verbose = False
    )
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_bids
    print("Deleting raw_bids...")
    t_step = dt.now()
    del raw_bids
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Downsampling the SSS file
    print("Downsampling the data...")
    t_step = dt.now()
    raw_downsampled_sss = raw_sss.resample(sfreq = 200) # automatic low-pass at 100 Hz
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")
    
    # Deleting raw_sss
    print("Deleting raw_sss...")
    t_step = dt.now()
    del raw_sss
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Write the downsampled data
    print("Writing the filtered data...")
    t_step = dt.now()
    raw_downsampled_sss.save(subject.path_downsampled_sss, overwrite = True, verbose = False)  # Saves in BIDS format
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Apply a band-pass filter
    print("Filter the data for the ICA...")
    t_step = dt.now()
    raw_for_ica = raw_downsampled_sss.copy().filter(l_freq = 0.5, h_freq = 30, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_downsampled_sss
    print("Deleting raw_downsampled_sss...")
    t_step = dt.now()
    del raw_downsampled_sss
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Create the ICA object
    print("Create the ICA...")
    t_step = dt.now()
    ica = mne.preprocessing.ICA(  # Create an ICA object
        n_components = 20,  # Define how many components we want in our ICA
        random_state = 97,   # Define a seed for the randomness generator to get consistent results
        method = "picard",  # Makes the ICA faster
        fit_params = dict(ortho = True, extended = True),
        verbose = False
    )
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Fit the ICA to our data
    print("Fit the ICA...")
    t_step = dt.now()
    ica.fit(raw_for_ica, picks = 'meg', verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Save the ICA
    print("Save the ICA...")
    t_step = dt.now()
    ica.save(subject.path_ica_solution, overwrite = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_for_ica
    print("Deleting raw_for_ica...")
    t_step = dt.now()
    del raw_for_ica
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")

## Step 4
* Find the HEOG, VEOG and ECG channels

<span style="color:#ffcc00">Manual step</span><br>
This step will be skipped if the channels are already defined in the dictionary in step 0. Otherwise, add an entry to the dictionary in step 0.

In [ ]:
mne.viz.set_browser_backend("qt")

timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    if subject.name not in eog_ecg_channels.keys():

        # Load raw for ICA data
        print("Loading the raw data...")
        t_step = dt.now()
        raw_for_ica = mne.io.read_raw_fif(subject.path_downsampled_sss, preload = True).filter(l_freq = 0.5, h_freq = 30)
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        # Load the ICA
        print("Loading the ICA...")
        t_step = dt.now()
        ica = mne.preprocessing.read_ica(subject.path_ica_solution)
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        heog = default_channels["HEOG"]  # Default: E0G061
        veog = default_channels["VEOG"]  # Default: E0G062
        ecg = default_channels["ECG"]  # Default: ECG063

        # Check using this line
        print("Plotting the picked channels for EOG and ECG...")
        t_step = dt.now()
        channels_of_interest = raw_for_ica.copy().pick_channels([heog, veog, ecg])
        channels_of_interest.plot(block = True, title = subject.name)
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        # Plot all the channels in case they are not the good ones
        print("Plotting all the channels in case the selected EOG and ECG are not the good ones...")
        t_step = dt.now()
        raw_for_ica.plot(overview_mode = "hidden", block = True, title = subject.name)
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        # Deleting raw_for_ica
        print("Deleting raw_for_ica...")
        t_step = dt.now()
        del raw_for_ica
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

    else:
        print("Channels already defined for this subject:")
        print(f"\tHEOG: {eog_ecg_channels[subject.name]['HEOG']}")
        print(f"\tVEOG: {eog_ecg_channels[subject.name]['VEOG']}")
        print(f"\tECG: {eog_ecg_channels[subject.name]['ECG']}")
        
    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")

## Step 5
* Find the ICA components

<span style="color:#99cc00">Semi-automatic step, approx. 4 min per subject</span><br>

Run this step, then check the outputs to decide which components to keep.

You can decide which plots you want to display for each subject on step 0.

This step can be skipped if the components are already defined in the dictionary in step 0.

In [ ]:
mne.viz.set_browser_backend("qt")

timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    # Load raw for ICA data
    print("Loading the raw data...")
    t_step = dt.now()
    raw_for_ica = mne.io.read_raw_fif(subject.path_downsampled_sss, preload = True, verbose = False).filter(l_freq = 0.5, h_freq = 30, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Load the ICA
    print("Loading the ICA...")
    t_step = dt.now()
    ica = mne.preprocessing.read_ica(subject.path_ica_solution, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Use subject
    if subject.name in eog_ecg_channels.keys():
        print("Getting the subject's custom EOG/ECG channels:")
        heog = eog_ecg_channels[subject.name]["HEOG"]  # Default: E0G061
        veog = eog_ecg_channels[subject.name]["VEOG"]  # Default: E0G062
        ecg = eog_ecg_channels[subject.name]["ECG"]  # Default: ECG063

    else:
        print("Getting the default EOG/ECG channels:")
        heog = default_channels["HEOG"]  # Default: E0G061
        veog = default_channels["VEOG"]  # Default: E0G062
        ecg = default_channels["ECG"]  # Default: ECG063

    print(f"\tHEOG: {heog}")
    print(f"\tVEOG: {veog}")
    print(f"\tECG: {ecg}")

    if subject.name in ica_components.keys():
        print(f"Automatically setting the saved components to remove: {', '.join([str(i) for i in ica_components[subject.name]])}")
        ica.exclude = ica_components[subject.name]

    else:
        # Analyze EOG/ECG channels
        print("Getting the HEOG components from the ICA...")
        t_step = dt.now()
        heog_indices, heog_scores = ica.find_bads_eog(raw_for_ica, ch_name = heog, verbose = False)
        print(f"\tComponents calculated: {', '.join([str(i) for i in heog_indices])}")
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        print("Getting the VEOG components from the ICA...")
        t_step = dt.now()
        veog_indices, veog_scores = ica.find_bads_eog(raw_for_ica, ch_name = veog, verbose = False)
        print(f"\tComponents calculated: {', '.join([str(i) for i in veog_indices])}")
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        print("Getting the ECG components from the ICA...")
        t_step = dt.now()
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw_for_ica, ch_name = ecg, verbose = False)
        print(f"\tComponents calculated: {', '.join([str(i) for i in ecg_indices])}")
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        mne.viz.set_browser_backend("matplotlib")

        # Reject ICAs containing the EOG and ECG information from all channels
        ica.exclude = list(set(veog_indices + heog_indices + ecg_indices))
        print(f"Combined rejected ICAS: {str(ica.exclude)}")

        if plot_components_locations:
            print(f"Plotting the components' locations...")
            t_step = dt.now()
            plot = ica.plot_components(title = subject.name)
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")

        if plot_components_time_course:
            print(f"Plotting the time course of the ICA-generated components...")
            t_step = dt.now()
            mne.viz.set_browser_backend("matplotlib")
            raw_for_ica.load_data()
            ica.plot_sources(raw_for_ica, start = 0., stop = 30., show_scrollbars = False, title = subject.name)
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")

        if plot_ecg_eog:
            print("Plotting the EOG and ECG channels...")
            t_step = dt.now()
            channels_of_interest = raw_for_ica.copy().pick_channels([heog, veog, ecg])
            channels_of_interest.plot(start = 0., duration = 30., show_scrollbars = False, title = subject.name, scalings = {"eog": 1e-3, "ecg": 5e-4, "eeg": 5e-4})
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")

        if plot_components_scores:
            print(f"Plotting all the components scores for EOG/ECG...")
            t_step = dt.now()

            plt.rcParams["figure.figsize"] = (15, 3)
            fig, axes = plt.subplots(1, 3)
            plt.setp(axes, xticks=[i for i in range(ica.n_components)], ylim=[0, 1])

            plt.sca(axes[0])
            plt.bar([i for i in range(ica.n_components)], np.abs(heog_scores), color=["blue" if i in heog_indices else "black" for i in range(ica.n_components)])
            plt.title(subject.name + " HEOG")

            plt.sca(axes[1])
            plt.bar([i for i in range(ica.n_components)], np.abs(veog_scores), color=["green" if i in veog_indices else "black" for i in range(ica.n_components)])
            plt.title(subject.name + " VEOG")

            plt.sca(axes[2])
            plt.bar([i for i in range(ica.n_components)], np.abs(ecg_scores), color=["red" if i in ecg_indices else "black" for i in range(ica.n_components)])
            plt.title(subject.name + " ECG")

            fig.tight_layout()
            plt.show()
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")

        if plot_explained_variance:
            print(f"Plotting the explained variance for each component...")
            t_step = dt.now()

            # Unitize variances explained by PCA components, so the values sum to 1
            pca_explained_variances = ica.pca_explained_variance_ / ica.pca_explained_variance_.sum()

            # Now extract the variances for those components that were used to perform ICA
            ica_explained_variances = pca_explained_variances[:ica.n_components_]

            axes = plt.gca()
            axes.set_xticks([i for i in range(ica.n_components)])
            axes.set_ylim([0, 1])
            axes.yaxis.set_major_formatter(mtick.PercentFormatter())

            plt.bar([i for i in range(ica.n_components)], ica_explained_variances)

            for i in range(ica.n_components):
                    plt.text(i,  # x position
                            ica_explained_variances[i] + .05,  # y position
                            str(round(ica_explained_variances[i]*100, 2)) + " %",  # Text to plot
                            ha = 'center')
            plt.title(subject.name)
            plt.show()
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")

        if plot_excluded_components_and_original:
            print(f"Plotting the first 30 seconds of the original EOG/ECG channels and the removed components...")
            t_step = dt.now()
            plot = ica.plot_sources(raw_for_ica, picks = ica.exclude, start = 0., stop = 30., show_scrollbars = False, title = subject.name)
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")
            
        if plot_overlay:
            print(f"Overlaying the components and the original channels...")
            t_step = dt.now()

            plt.rcParams["figure.figsize"] = (15, 5)
            plot = ica.plot_overlay(
                raw_for_ica,  # The original data
                exclude = ica.exclude,  # The ICA component to remove
                picks = 'meg',   # The channels to display
                start = 0.,  # First timestamp of the display window (seconds)
                stop = 30.,  # Last timestamp of the display window (seconds)
                title = subject.name,
            )
            if show_timings:
                print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_for_ica
    print("Deleting raw_for_ica...")
    t_step = dt.now()
    del raw_for_ica
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")

## Step 6
* Reject the components (based on the components set on step 0)
* Save the ICA-filtered data
* Save the pictures showing the ICA components
* Save the ICA in eelbrain format
* Find the closest empty room recording
* Create the BIDS for the empty room
* Automatically detect the bad channels for the empty room

<span style="color:#99cc00">Fully automatic step, approx. 20 min per subject</span><br>

In [ ]:
timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    # Load raw for ICA data
    print("Loading the raw data...")
    t_step = dt.now()
    raw_downsampled_sss = mne.io.read_raw_fif(subject.path_downsampled_sss, preload = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Load the ICA
    print("Loading the ICA...")
    t_step = dt.now()
    ica = mne.preprocessing.read_ica(subject.path_ica_solution, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Reject ICAs containing the EOG and ECG information from all channels
    if subject.name in ica_components.keys():
        ica.exclude = ica_components[subject.name]
        print(f"Rejected ICAS: {str(ica.exclude)}")
    else:
        raise Exception("""Please define the components to exclude for this subject on step 0 first (dictionary "ica_components")""")

    # Reject the components
    print(f"Removing the components from the raw data...")
    t_step = dt.now()
    raw_filtered_ica = ica.apply(raw_downsampled_sss, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_downsampled_sss
    print("Deleting raw_downsampled_sss...")
    t_step = dt.now()
    del raw_downsampled_sss
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Save the ICA-filtered data
    print(f"Saving the ICA-filtered data...")
    t_step = dt.now()
    raw_filtered_ica.save(subject.path_ica, overwrite = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Save the ICA in eelbrain format
    print(f"\tSaving the ICA-filtered data in eelbrain format...")
    t_step = dt.now()
    raw_eelbrain = raw_filtered_ica.copy().pick(['meg','stim', 'misc', 'chpi'])  # We remove the EEG channels (in the case they exist) as they conflict with eelbrain
    raw_eelbrain.save(subject.path_eelbrain, overwrite = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if save_figures:

        # Use subject
        if subject.name in eog_ecg_channels.keys():
            print("Getting the subject's custom EOG/ECG channels:")
            heog = eog_ecg_channels[subject.name]["HEOG"]  # Default: E0G061
            veog = eog_ecg_channels[subject.name]["VEOG"]  # Default: E0G062
            ecg = eog_ecg_channels[subject.name]["ECG"]  # Default: ECG063

        else:
            print("Getting the default EOG/ECG channels:")
            heog = default_channels["HEOG"]  # Default: E0G061
            veog = default_channels["VEOG"]  # Default: E0G062
            ecg = default_channels["ECG"]  # Default: ECG063

        # Analyze EOG/ECG channels
        print("Getting the HEOG components from the ICA...")
        t_step = dt.now()
        heog_indices, heog_scores = ica.find_bads_eog(raw_filtered_ica, ch_name = heog, verbose = False)
        print(f"\tComponents calculated: {', '.join([str(i) for i in heog_indices])}")
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        print("Getting the VEOG components from the ICA...")
        t_step = dt.now()
        veog_indices, veog_scores = ica.find_bads_eog(raw_filtered_ica, ch_name = veog, verbose = False)
        print(f"\tComponents calculated: {', '.join([str(i) for i in veog_indices])}")
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        print("Getting the ECG components from the ICA...")
        t_step = dt.now()
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw_filtered_ica, ch_name = ecg, verbose = False)
        print(f"\tComponents calculated: {', '.join([str(i) for i in ecg_indices])}")
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        print(f"Saving the figures...")
        t_step = dt.now()

        # All components
        print(f"\tSaving the figure containing all the components...")
        f1 = ica.plot_components(show=False)
        path_ica_all_components = op.join(subject.path_derivatives.directory, subject.path_derivatives.basename.replace("meg", "ica_all_components") + ".png")
        f1.savefig(path_ica_all_components, dpi=300)

        # Rejected components
        print(f"\tSaving the figure containing the rejected components...")
        f2 = ica.plot_components(picks=ica.exclude, show=False)
        path_ica_rejected_components = op.join(subject.path_derivatives.directory, subject.path_derivatives.basename.replace("meg", "ica_rejected_components") + ".png")
        f2.savefig(path_ica_rejected_components, dpi=300)

        # HEOG channel
        print(f"\tSaving the figure containing the HEOG scores...")
        f3 = ica.plot_scores(heog_scores, show=False)
        path_ica_heog = op.join(subject.path_derivatives.directory, subject.path_derivatives.basename.replace("meg", "heog_scores") + ".png")
        f3.savefig(path_ica_heog, dpi=300)

        # VEOG channel
        print(f"\tSaving the figure containing the VEOG scores...")
        f4 = ica.plot_scores(veog_scores, show=False)
        path_ica_veog = op.join(subject.path_derivatives.directory, subject.path_derivatives.basename.replace("meg", "veog_scores") + ".png")
        f4.savefig(path_ica_veog, dpi=300)

        # ECG channel
        print(f"\tSaving the figure containing the ECG scores...")
        f5 = ica.plot_scores(ecg_scores, show=False)
        path_ica_ecg = op.join(subject.path_derivatives.directory, subject.path_derivatives.basename.replace("meg", "ecg_scores") + ".png")
        f5.savefig(path_ica_ecg, dpi=300)

        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_filtered_ica
    print("Deleting raw_filtered_ica...")
    t_step = dt.now()
    del raw_filtered_ica
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    print("Finding the closest empty room data...")
    t_step = dt.now()
    info = mne.io.read_info(subject.raw_fif_name, verbose = False)
    date_measurement = info["meas_date"].replace(tzinfo = None)
    print(f"\tMeasurement date: {date_measurement.strftime('%A %d %B %Y %H:%M:%S')}")

    # Get all of the recordings folders
    subfolders_empty_room = os.listdir(path_empty_room)
    subfolders_empty_room.remove("other")

    empty_room_recordings = []
    for subfolder in subfolders_empty_room:
        subfolder_recordings = os.listdir(op.join(path_empty_room, subfolder))
        for recording in subfolder_recordings:
            if op.isdir(op.join(path_empty_room, subfolder, recording)):
                empty_room_recordings.append(subfolder + "/" + recording)

    # Find the smallest difference of days
    smallest_diff = None
    selected_recording = None
    for folder in empty_room_recordings:
        recording = folder.split("/")[1]
        if recording != ".directory":
            date_empty_room = dt(int(recording[0:2]) + 2000, int(recording[2:4]), int(recording[4:6]))  # Turn the folder name into a date
            day_diff = date_measurement - date_empty_room
            if selected_recording is None or abs(smallest_diff) > abs(day_diff):  # If we find a smaller difference of days than the previous, we set it
                smallest_diff = day_diff
                selected_recording = folder

    # Print the selected empty room
    folder_name = selected_recording.split("/")[1]
    empty_room_date = dt(int(folder_name[0:2]) + 2000, int(folder_name[2:4]), int(folder_name[4:6])).strftime("%A %d %B %Y")
    if smallest_diff.days > 0:
        print(f"\tSelected empty room recording: {selected_recording}, recorded {empty_room_date} ({smallest_diff.days} day(s) before the subject recording).")
    elif smallest_diff.days < 0:
        print(f"\tSelected empty room recording: {selected_recording}, recorded {empty_room_date} ({abs(smallest_diff.days)} day(s) after the subject recording).")
    else:
        print(f"\tSelected empty room recording: {selected_recording}, recorded {empty_room_date} (the same day of the subject recording).")

    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    path_selected_er = op.join(path_empty_room, selected_recording)
    path_empty_room_fif = op.join(path_selected_er, os.listdir(path_selected_er)[0])
    print(f"\tPath: {path_empty_room_fif}")

    # Read the FIF file
    print("Reading the empty room FIF file...")
    t_step = dt.now()
    empty_room_fif = mne.io.read_raw_fif(path_empty_room_fif, preload = False, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # We remove any digitization point that could be saved from a previous session, so we use a proper empty room
    empty_room_fif.set_montage(None)

    # Write the BIDS files
    print("Writing BIDS...")
    t_step = dt.now()
    write_raw_bids(raw = empty_room_fif, bids_path = subject.path_bids_empty_room, overwrite = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting empty_room_fif
    print("Deleting empty_room_fif...")
    t_step = dt.now()
    del empty_room_fif
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Open the BIDS data
    print("Opening BIDS data...")
    t_step = dt.now()
    raw_bids_empty_room = read_raw_bids(subject.path_bids_empty_room, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Define the "bads" channels to being empty
    raw_bids_empty_room.info['bads'] = []

    # Run the automatic detection
    print("Running the automatic detection of bad channels...")
    t_step = dt.now()
    auto_noisy_channels_er, auto_flat_channels_er, auto_scores_er = mne.preprocessing.find_bad_channels_maxwell(
        raw_bids_empty_room,  # Raw data to process
        cross_talk = crosstalk_file, 
        calibration = fine_cal_file,
        return_scores = True, 
        coord_frame = "meg",  # Necessary for empty room recordings
        duration = 30,  # Duration of each window (in seconds)
        min_count = 10,  # Number of window appearances to be counted as bad channel
        verbose = False,
        )
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    print(f"Flat channels: {str(auto_flat_channels_er)}")
    print(f"Bad channels: {str(auto_noisy_channels_er)}")

    raw_bids_empty_room.info['bads'] = auto_flat_channels_er + auto_noisy_channels_er

    print("Writing the BIDS with the detected bad channels info...")
    t_step = dt.now()
    mark_channels(subject.path_bids_empty_room, ch_names = raw_bids_empty_room.info['bads'], status = "bad", verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_bids_empty_room
    print("Deleting raw_bids_empty_room...")
    t_step = dt.now()
    del raw_bids_empty_room
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")

## Step 7

* Manual channel detection: click on the channels you want to remove.

<span style="color:#ffcc00">Manual step</span><br>
If the dictionary `bad_channels_subject_er` in step 0 contains an entry with the subject name, this step becomes automatic.

In [ ]:
mne.viz.set_browser_backend("qt")

timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    # Open the BIDS data
    print("Opening BIDS data...")
    t_step = dt.now()
    raw_bids_empty_room = read_raw_bids(subject.path_bids_empty_room, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    print("Automatically detected bad channels: ")
    print(", ".join(raw_bids_empty_room.info['bads']))

    # We skip it if we saved the bad channels in bad_channels_subject
    if subject.name in bad_channels_subject_er.keys():

        raw_bids_empty_room.info['bads'] += bad_channels_subject_er[subject.name]

        print("Bad channels after adding the ones saved in bad_channels_subject_er: ")
        print(", ".join(raw_bids_empty_room.info['bads']))

    else:
        print("Opening the GUI to manually mark bad channels...")
        t_step = dt.now()

        automatic_channels = []
        for channel in raw_bids_empty_room.info['bads']:
            automatic_channels.append(channel)
        
        raw_bids_empty_room.plot(lowpass = 30, highpass = 1, overview_mode = "hidden", title = subject.name, block = True)
        if show_timings:
            print(f"\tPerformed in {dt.now() - t_step}")

        print("Manually added channels: ")
        manual_channels = list(set(raw_bids_empty_room.info['bads']) - set(automatic_channels))
        print(", ".join(manual_channels))

        print("Bad channels after manual processing: ")
        print(", ".join(raw_bids_empty_room.info['bads']))

    # Write the BIDS data
    print("Writing the BIDS with the detected bad channels info...")
    t_step = dt.now()
    mark_channels(subject.path_bids_empty_room, ch_names = raw_bids_empty_room.info['bads'], status = "bad", verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_bids_empty_room
    print("Deleting raw_bids_empty_room...")
    t_step = dt.now()
    del raw_bids_empty_room
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")


## Step 8
* Run the Maxfilter
* Save the filtered data

<span style="color:#99cc00">Fully automatic step, approx. 5 min per subject</span><br>

In [ ]:
# mne.set_config("MNE_USE_NUMBA", "true")

timings = {}

for subject in subjects:

    t_begin = dt.now()
    subject.print_name()

    # Open the BIDS data
    print("Opening BIDS data...")
    t_step = dt.now()
    raw_bids_empty_room = read_raw_bids(subject.path_bids_empty_room, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Run the Maxfilter
    print("Running the Maxfilter...")
    t_step = dt.now()
    raw_er_sss = mne.preprocessing.maxwell_filter(
        raw_bids_empty_room, 
        cross_talk = crosstalk_file, 
        calibration = fine_cal_file,
        coord_frame = "meg",  # Necessary for empty room recordings
        st_duration = 10, # Duration of the buffer (in seconds) 
        st_correlation = 0.98,  # Correlation limit between inner and outer subspaces used to reject overlapping intersecting inner/outer signals during spatiotemporal SSS.
        verbose = False
    )
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_bids_empty_room
    print("Deleting raw_bids_empty_room...")
    t_step = dt.now()
    del raw_bids_empty_room
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Downsampling the SSS file
    print("Downsampling the data...")
    t_step = dt.now()
    raw_er_downsampled_sss = raw_er_sss.resample(sfreq = 200) # automatic low-pass at 100 Hz
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")
    
    # Deleting raw_er_sss
    print("Deleting raw_er_sss...")
    t_step = dt.now()
    del raw_er_sss
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")
    
    print("Writing the filtered data...")
    t_step = dt.now()
    raw_er_downsampled_sss.save(subject.path_downsampled_sss_er, overwrite = True, verbose = False)  # Saves in BIDS format
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    print("Writing the filtered data for eelbrain...")
    t_step = dt.now()
    raw_er_eelbrain = raw_er_downsampled_sss.copy().pick(['meg','stim'])  # We remove the EEG channels (in the case they exist) as they conflict with eelbrain
    raw_er_eelbrain.save(subject.path_eelbrain_er, overwrite = True, verbose = False)
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    # Deleting raw_er_downsampled_sss
    print("Deleting raw_er_downsampled_sss...")
    t_step = dt.now()
    del raw_er_downsampled_sss
    if show_timings:
        print(f"\tPerformed in {dt.now() - t_step}")

    if show_timings:
        timings[subject.name] = dt.now() - t_begin
        print(f"Full step performed in {dt.now() - t_begin}")

if show_timings:
    print("Timings for each subject:")
    total = td(0)
    for subject in timings.keys():
        print(f"\t{subject}\t{timings[subject]}")
        total += timings[subject]
    print(f"\tAverage:\t{total/(len(subjects))}")